# Задача анализа тональности отзывов

# Подготовка библиотек и датасетов

In [ ]:
!pip install PyDrive
!pip install pymorphy2
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 8.2MB 4.9MB/s 
Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 32.27 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3031704 sha256=14f78f7a50ee4964f64c1c731d27d96f4baac1488987bd3955f5c4a278bcf9e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-hffkz_tq/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext


In [ ]:
! pip install RestrictedPython

In [ ]:
# базовые необходимые библиотеки
import pandas as pd
import numpy as np
import string
import re   # библиотека для регулярных выражений
from bs4 import BeautifulSoup # библиотека для парсинга xml

Выберем данные, с которыми будем работать для примера конвеера.
(Те, кто предпочитает работать с английским языком, могут использовать набор данных sms_spam, он есть в папке Data).

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1j-DhO_XD5EqzOqVSR4Wz1kNzhLmpeTkZ"}) 
downloaded.GetContentFile('women-clothing-accessories.csv')
data_reviews = pd.read_csv('women-clothing-accessories.csv', sep='\t', usecols=[0, 1])

In [ ]:
data_reviews.head()

,review,sentiment
0,качество плохое пошив ужасный (горловина напер...,negative
1,"Товар отдали другому человеку, я не получила п...",negative
2,"Ужасная синтетика! Тонкая, ничего общего с пре...",negative
3,"товар не пришел, продавец продлил защиту без м...",negative
4,"Кофточка голая синтетика, носить не возможно.",negative


In [ ]:
downloaded = drive.CreateFile({'id': "1IKfQNbkGADSFUdyPdVjQS-ucFhOiT6sM"}) 
downloaded.GetContentFile('sms_spam.csv')
data_sms = pd.read_csv('sms_spam.csv', usecols=[1, 0])

In [ ]:
data_sms.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...


In [ ]:
del(downloaded)

Scikit-Learn — расширение для библиотеки SciPy (Scientific Python), предоставляющее прикладной интерфейс (API) для обобщенного машинного обучения. Основанное на Cython с поддержкой высокопроизводительных библиотек на C, таких как LAPACK, LibSVM, Boost и других, расширение Scikit-Learn сочетает высокую производительность с простотой использования методов анализа наборов данных малого и среднего размера. 
Это расширение предоставляет единый интерфейс дл многих моделей регрессии, классификации, кластеризации и уменьшения размерности, а также утилиты для перекрестной проверки и настройки гиперпараметров.


MultinomialNB - Наивный байесовский классификатор для полиномиальных моделей. Он работает с дискретными хар-ками, что позоволяет подсчитывать слова для классификации. Полиномиальное распределение обычно требует целочисленного подсчета признаков.



In [ ]:
from sklearn.metrics import * 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier # можно заменить на другой классификатор
from sklearn.naive_bayes import MultinomialNB # наивный байесовский классификатор
from sklearn.feature_extraction.text import CountVectorizer # модель "мешка слов", см. далее
from sklearn.feature_extraction.text import TfidfVectorizer

NLTK (Natural Language Tool-Kit — пакет инструментов для обработки
естественного языка) — это «батарейки в комплекте», ресурс для обработ­ки естественного языка. Пакет содержит корпусы, лексические ресурсы, грамматики, алгоритмы обработки языков и предварительно обученные модели, позволяя программистам на Python быстро приступать к обработке текстовых данных на разных естественных языках.

In [ ]:
import nltk
from nltk import ngrams, tokenize
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
# готовые токенизаторы библиотеки nltk
nltk.download('punkt') # пунктуация для правильной работы токенизатора
from nltk.corpus import stopwords # стоп-слова из NLTK
nltk.download('stopwords')
import nltk.data # библиотека Natural Language Toolkit

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Gensim — надежная, эффективная и простая в использовании библиотека, главной целью которой является семантическое моделирование текста без учителя. Первоначально создававшаяся для поиска сходств в документах, в настоящее время эта библиотека предоставляет тематическое моделирование для методов латентно-семантического анализа и включает в себя другие библиотеки машинного обучения без учителя, такие как word2vec.

In [ ]:
from pymorphy2 import MorphAnalyzer

In [ ]:
import gensim # библиотека для загрузки и использвоания моделй w2v
from gensim.models import word2vec # непосредственно методы w2v

In [ ]:
import urllib.request # библиотека для скачивания данных
import fasttext

# Постановка задачи

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
токенизация
* приведение к нижнему регистру
* удаление стоп-слов
* лемматизация
* векторизация (с настройкой гиперпараметров)
* построение модели
* оценка качества модели

Обязательно использование векторайзеров:
* мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
* tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
* символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать 
* наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте 
* precision, 
* recall, 
* f1-score и 
* accuracy. 

Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в ячейках будут значения этих метрик для соответсвующих векторайзеров.


# Ход работы

In [ ]:
noise = stopwords.words('english')+stopwords.words('russian')
for i in string.punctuation:
  noise.append(i)
noise.append('это')
print(sorted(noise))

['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'oursel

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
data_reviews['review'][0]

'качество плохое пошив ужасный (горловина наперекос) Фото не соответствует Ткань ужасная рисунок блеклый маленький рукав не такой УЖАС!!!!! не стоит за такие деньги г.......'

In [ ]:
# выжно сбросить индекс, чтобы можно было заменить колонку - замена по индексу
data_reviews_wo_punct = data_reviews.copy().sample(500).reset_index(drop=True)
# data_reviews_wo_punct = data_reviews.copy()
data_reviews_wo_punct

,review,sentiment
0,долгое ожтдание в итоге нет товара деньги верн...,negative
1,"Редкостное дерьмо, фото вообще не соответствуе...",negative
2,"Качество хорошее, с размером не угадала, разме...",positive
3,"заказ, к сожалению, я так и не получила!хотя р...",neautral
4,нитки лезут( носить невозможно особенно если х...,neautral
...,...,...
495,"шли чуть меньше месяца, все отлично, на 38 хор...",positive
496,"платье на раз,тонкое,липкое...симпатичное...ра...",neautral
497,порвались в первый же день на 170-55,neautral
498,Заказала самый большой размер. По таблице разм...,neautral


In [ ]:
data_reviews_wo_punct['review'][0].split()

['долгое',
 'ожтдание',
 'в',
 'итоге',
 'нет',
 'товара',
 'деньги',
 'вернули',
 'после',
 'вмешательства',
 'экспресс']

! деление на униграммы (n-граммы) показывает, что перед удаление пунктуации нужно убедиться, что оставшееся слово существует

In [ ]:
# в некоторых ситуациях списковые включения имеют побочные эффекты, которые могут
# изменить возвращаемое функцией состояние, чо и происходит в случае написания:
pd.DataFrame([
              review.replace(ch,"") 
              for ch in string.punctuation 
              for review in data_reviews['review']
              ])
# исполнение примера показывает, что функция не воспроизводит изменения

In [ ]:
# пример для демонстрации работы
for review in data_reviews['review']:
  for ch in string.punctuation:
    review = review.replace(ch,"")
  print(review)

In [ ]:
pymorphy2_analyzer = MorphAnalyzer()

In [ ]:
sentence = data_reviews_wo_punct['review'][0]
for word in word_tokenize(sentence):
  #print(word, type(word), len(pymorphy2_analyzer.parse(word))#, pymorphy2_analyzer.parse(word)
  #)
  word_forms = pymorphy2_analyzer.parse(word)
  if len(word_forms)>0:
    print(word, '--',word_forms[0].normal_form)

долгое -- долгий
ожтдание -- ожтдание
в -- в
итоге -- итог
нет -- нет
товара -- товар
деньги -- деньга
вернули -- вернуть
после -- после
вмешательства -- вмешательство
экспресс -- экспресс


In [ ]:
def lemmatization(sentence):
  sentence = [
              pymorphy2_analyzer.parse(word)[0].normal_form 
              if len(pymorphy2_analyzer.parse(word))>0
              else word
              for word in sentence.split()]
  space_ch = " "
  sentence = space_ch.join(sentence)
  return sentence

def replace(stri, to):
  for ch in string.punctuation:
    stri = stri.replace(ch, to)
  return stri

review_col = pd.DataFrame([
                           lemmatization(replace(review, " "))
                           for review in data_reviews_wo_punct['review']
                           ])
review_col

,0
0,долгий ожтдание в итог нет товар деньга вернут...
1,редкостный дерьмо фото вообще не соответствова...
2,качество хороший с размер не угадать размер гр...
3,заказ к сожаление я так и не получить хотя ран...
4,нитка лезть носить невозможно особенно если хо...
...,...
495,идти чуть маленький месяц всё отлично на 38 хо...
496,платье на раз тонкий липкий симпатичный раз дв...
497,порваться в первый же день на 170 55
498,заказать самый большой размер по таблица разме...


In [ ]:
data_reviews_wo_punct['review'] = review_col
data_reviews_wo_punct

,review,sentiment
0,долгий ожтдание в итог нет товар деньга вернут...,negative
1,редкостный дерьмо фото вообще не соответствова...,negative
2,качество хороший с размер не угадать размер гр...,positive
3,заказ к сожаление я так и не получить хотя ран...,neautral
4,нитка лезть носить невозможно особенно если хо...,neautral
...,...,...
495,идти чуть маленький месяц всё отлично на 38 хо...,positive
496,платье на раз тонкий липкий симпатичный раз дв...,neautral
497,порваться в первый же день на 170 55,neautral
498,заказать самый большой размер по таблица разме...,neautral


* добавлено удаление пунктуации
* оптимизированно
* добавлена лемматизация
* оптимизированно х3
* // важен порядок: сначала замена пунктуации на пробел, затем лемматизация, тк иначе слова в некоторых примерах склеиваются
* // лемматизация переводит слова в нижний регистр
* использовано шумоподавление на основе массива шума на 2х языках: английском и русском




In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    data_reviews_wo_punct.review, 
    data_reviews_wo_punct.sentiment, train_size = 0.7)

In [ ]:
data_reviews_wo_punct.sentiment.value_counts()

neautral    175
positive    168
negative    157
Name: sentiment, dtype: int64

In [ ]:
y_train.value_counts()

positive    120
neautral    120
negative    110
Name: sentiment, dtype: int64

In [ ]:
# здесь осуществяется подбор размера грамм

In [ ]:
# здесь нужно учитывать шум

In [ ]:
# векторизация мешок слов

bow_vectorizer = CountVectorizer(ngram_range=(2, 4), max_df=0.1, min_df=4, max_features=50, stop_words=noise)
#bow_vectorizer = CountVectorizer(ngram_range=(1, 1))
vectorized_x_train = bow_vectorizer.fit_transform(x_train)
list(bow_vectorizer.vocabulary_.items())[:10], vectorized_x_train.shape, x_train.shape

([('ткань очень', 43),
  ('доставка быстрый', 8),
  ('48 50', 2),
  ('ждать месяц', 10),
  ('прийти деньга', 25),
  ('деньга вернуть', 7),
  ('прийти деньга вернуть', 26),
  ('качество хороший', 14),
  ('прийти быстро', 24),
  ('42 44', 0)],
 (350, 50),
 (350,))

In [ ]:
NBclf = MultinomialNB()

In [ ]:
NBclf.fit(vectorized_x_train, y_train)
vectorized_x_test = bow_vectorizer.transform(x_test)
pred = NBclf.predict(vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.42      0.76      0.54        55
    negative       0.53      0.17      0.26        47
    positive       0.66      0.48      0.55        48

    accuracy                           0.49       150
   macro avg       0.54      0.47      0.45       150
weighted avg       0.53      0.49      0.46       150



In [ ]:
BoW_features = bow_vectorizer.get_feature_names()

In [ ]:
# векторизация TD-IDF

tfidf_vectorizer = TfidfVectorizer(ngram_range=(2, 4), max_df=0.4, min_df=2, max_features=50, stop_words=noise)
# tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
# обучаем его и применяем к x_train
tfidf_vectorized_x_train = tfidf_vectorizer.fit_transform(x_train)
# инициализируем и обучаем классификатор
NBclf.fit(tfidf_vectorized_x_train, y_train)
# применяем обученный векторизатор к тестовым данным
tfidf_vectorized_x_test = tfidf_vectorizer.transform(x_test)
pred = NBclf.predict(tfidf_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.41      0.76      0.54        55
    negative       0.56      0.19      0.29        47
    positive       0.66      0.44      0.53        48

    accuracy                           0.48       150
   macro avg       0.54      0.46      0.45       150
weighted avg       0.54      0.48      0.45       150



In [ ]:
tfidf_features = tfidf_vectorizer.get_feature_names()

In [ ]:
# векторизатор на основе символьных n-грамм

char_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 4), max_df=0.1, min_df=4, max_features=50, stop_words=noise)
char_vectorized_x_train = char_vectorizer.fit_transform(x_train)
NBclf.fit(char_vectorized_x_train, y_train)
char_vectorized_x_test = char_vectorizer.transform(x_test)
pred = NBclf.predict(char_vectorized_x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

    neautral       0.37      0.51      0.43        55
    negative       0.39      0.28      0.33        47
    positive       0.45      0.40      0.42        48

    accuracy                           0.40       150
   macro avg       0.41      0.39      0.39       150
weighted avg       0.41      0.40      0.39       150



In [ ]:
char_features = char_vectorizer.get_feature_names()

In [ ]:
len(BoW_features), len(tfidf_features), len(char_features)

(50, 50, 50)

In [ ]:
d = {'BoW': BoW_features, 'TF-IDF': tfidf_features, 'Char': char_features}
features = pd.DataFrame(d)

In [ ]:
features

,BoW,TF-IDF,Char
0,42 44,42 44,s
1,44 размер,44 размер,вы
2,48 50,быстрый доставка,кр
3,48 размер,вернуть деньга,мал
4,быстрый доставка,деньга вернуть,по
5,вернуть деньга,доставка быстрый,пос
6,всё отлично,доставка неделя,се
7,деньга вернуть,ждать месяц,тк
8,доставка быстрый,заказ прийти,тка
9,доставка неделя,заказ прийти быстро,тов
